In [17]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.evaluation.qa import QAGenerateChain
from langchain.evaluation.qa import QAEvalChain

from langchain_ibm import ChatWatsonx, WatsonxEmbeddings
from ibm_watsonx_ai import Credentials

In [8]:
parameters = {
    "temperature": 0.0,
    "max_tokens": 1000,
}

llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_",
    project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd",
    params=parameters,
)

In [4]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"
)

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"
    )

In [7]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='latin1')
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch, embedding = embeddings
).from_loaders([loader])

c:\Users\m_gor\Desktop\agent1\.venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [10]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [14]:
example_gen_chain = QAGenerateChain.from_llm(llm)

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

print(len(new_examples))

new_examples[0]

c:\Users\m_gor\Desktop\agent1\.venv\lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


5


{'qa_pairs': {'query': "What type of material is used for the construction of the Women's Campside Oxfords, and what is the purpose of this material?",
  'answer': "The Women's Campside Oxfords are constructed using soft canvas material. This material is chosen for its broken-in feel and look, providing comfort from the first time the shoes are worn."}}

In [24]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

examples += [i['qa_pairs'] for i in new_examples]
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [25]:
## LLM assisted evaluation

In [26]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What type of material is used for the construction of the Women's Campside Oxfords, and what is the purpose of this material?",
  'answer': "The Women's Campside Oxfords are constructed using soft canvas material. This material is chosen for its broken-in feel and look, providing comfort from the first time the shoes are worn."},
 {'query': 'What are the dimensions of the Recycled Waterhog Dog Mat in medium size?',
  'answer': 'The dimensions of the Recycled Waterhog Dog Mat in medium size are 22.5" x 34.5".'},
 {'query': "What is the sun protection rating of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece has a UPF 50+ rating, providing the highest rated su

In [37]:
predictions = qa.apply(examples)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'].split("\n\n")[0])  #graded_outputs[i]['text'])
    print()



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.
Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What type of material is used for the construction of the Women's Campside Oxfords, and what is the purpose of this mate

In [36]:
graded_outputs[i]['results'].split("\n\n")[0]

'CORRECT'

In [32]:
graded_outputs[1]['results']

'CORRECT\n\nQUESTION: Who was the first person to walk on the moon?\nSTUDENT ANSWER: Neil Armstrong was the first person to walk on the moon.\nTRUE ANSWER: Neil Armstrong was the first person to walk on the moon.\nGRADE:'